In [3]:
# use VnCoreNLP word segmentation
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/home/phamson/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

FileNotFoundError: File "/home/phamson/VnCoreNLP/VnCoreNLP-1.1.1.jar" was not found, please check again.

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# # tạo label list tương ứng với bộ dữ liệu
# label_list = [
#     "O",       # Outside of a named entity
#     "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
#     "I-MISC",  # Miscellaneous entity
#     "B-PER",   # Beginning of a person's name right after another person's name
#     "I-PER",   # Person's name
#     "B-ORG",   # Beginning of an organisation right after another organisation
#     "I-ORG",   # Organisation
#     "B-LOC",   # Beginning of a location right after another location
#     "I-LOC"    # Location
# ]

In [12]:
from underthesea import sent_tokenize, word_tokenize
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)
phobert_ner = AutoModelForTokenClassification.from_pretrained('/opt/github/Phobert-Named-Entity-Reconigtion/phobert-ner-mck')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
# tạo label list tương ứng với bộ dữ liệu
# label_list = [
#     "B-CDT",
#     "B-DA",
#     "I-CDT",
#     "I-DA",
#     "O"

# ]

label_list = [
    "B-CNAME",
    "B-HNAME",
    "I-CNAME",
    "I-HNAME",
    "MCK",
    "O"
]

In [17]:
txt_file = '/home/phamson/data/real estate news/1.txt'

with open(txt_file, 'r') as file:
    for line in file:
        sentences = annotator.tokenize(line)
        for sentence in sentences:
            sequence = " ".join(sentence)  # tạo câu mới đã được tokenized 
            input_ids = torch.tensor([tokenizer.encode(sequence)])  # lấy id của các tokens tương ứng 
            # không dùng tokenize(decode(encode)), text sẽ bị lỗi khi tokenize do conflict với tokenizer mặc định
            tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(sequence))  # lấy các token để đánh tags 

            outputs = phobert_ner(input_ids).logits
            predictions = torch.argmax(outputs, dim=2)
            print(predictions)

            for i in [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
                print(i)
            print('---------------------------------------------')        

FileNotFoundError: [Errno 2] No such file or directory: '/home/phamson/data/real estate news/1.txt'

In [51]:
# text = "Ngày 27/4, UBND TPHCM tổ chức lễ công bố quyết định của Thủ tướng Chính phủ công nhận huyện Cần Giờ đạt chuẩn nông thôn mới năm 2020 và công nhận xã đảo Thạnh An là xã đảo thuộc TPHCM."

# text = "Chủ tịch UBND TPHCM Nguyễn Thành Phong nhấn mạnh: đây là sự kiện quan trọng đánh dấu bước phát triển mới của huyện Cần Giờ, mở ra nhiều cơ hội để thúc đẩy phát triển kinh tế trong thời gian tới."

# text = "Thạnh An là xã đảo của huyện cần Giờ cách trung tâm huyện khoảng 8km tính theo đường chim bay và cách trung tâm TPHCM khoảng 50 km. Tổng diện tích tự nhiên cùa xã là 13.130 ha, chiếm 18,6% diện tích tự nhiên cùa huyện (ảnh: Trung Kiên)."

# text = 'Đặc biệt, với việc công nhận Thạnh An là xã đảo, người dân Thạnh An sẽ được hưởng nhiều cơ chế chính sách hơn cũng như các điều kiện thuận lợi để thúc đẩy phát triển kinh tế và nhanh chóng bắt kịp các xã khác của huyện Cần Giờ.'
#
# "Cuối năm 2020, thành phố vui mừng đón nhận Cù lao Gò Gia được Chính phủ công nhận thuộc TPHCM sau hơn 40 năm chồng lấn địa giới hành chính với tỉnh Đồng Nai. Và nay, Thủ tướng Chính phủ công nhận huyện Cần Giờ đạt chuẩn nông thôn mới năm 2020 và công nhận xã Thạnh An là xã đảo thuộc TPHCM", Chủ tịch UBND TPHCM bày tỏ.\
# Theo người đứng đầu chính quyền thành phố, con đường phía trước của Cần Giờ đang mở ra với nhiều cơ hội và thách thức đan xen, đòi hỏi Đảng bộ, Chính quyền và Nhân dân huyện Cần Giờ phải chủ động, sáng tạo, quyết liệt và nỗ lực không ngừng vươn lên.'
# text = "Đặc_biệt , với việc công_nhận Thạnh_An là xã đảo , người_dân Thạnh_An sẽ được hưởng nhiều cơ_chế chính_sách hơn cũng như các điều_kiện thuận_lợi để thúc_đẩy phát_triển kinh_tế và nhanh_chóng bắt kịp các xã khác của huyện Cần_Giờ ."

# text = 'Đặc biệt, với việc công nhận Thạnh An là xã đảo, người dân Thạnh An sẽ được hưởng nhiều cơ chế chính sách hơn cũng như các điều kiện thuận lợi để thúc đẩy phát triển kinh tế và nhanh chóng bắt kịp các xã khác của huyện Cần Giờ.'
# text = "Ngày 27/4, UBND TPHCM tổ chức lễ công bố quyết định của Thủ tướng Chính phủ công nhận huyện Cần Giờ đạt chuẩn nông thôn mới năm 2020 và công nhận xã đảo Thạnh An là xã đảo thuộc TPHCM."
# text = "Chủ tịch UBND TPHCM Nguyễn Thành Phong nhấn mạnh: đây là sự kiện quan trọng đánh dấu bước phát triển mới của huyện Cần Giờ, mở ra nhiều cơ hội để thúc đẩy phát triển kinh tế trong thời gian tới."

# text = "dự án Sun Grand City New An Thới với chủ đầu tư là công ty Sungroup"
# text = "Sun Grand City New An Thới nằm tiếp giáp đường 975 tuyến giao thông huyết mạch nối liền sân bay Phú Quốc, thị trấn An Thới, đường Nguyễn Văn Cừ, đây cũng là tuyến đường trung tâm kết nối mọi dự án của tập đoàn Sun Group tại đảo Ngọc."
# text = "Thông tin mô tả\
# Bán đất chính chủ, sổ đỏ đầy đủ, đường 24m, khu đô thị Hà Phong, Tiền Phong, Mê Linh, Hà Nội.\
# Diện tích 160m2, giá 50 triệu/m², có thương lượng.\
# Khu Đô Thị mới Hà Phong thuộc quần thể ĐTM Bắc Thăng Long, dự án được quy hoạch đạt tiêu chuẩn của KĐT hiện đại với kết cấu hạ tầng tiên tiến và hoàn chỉnh.\
# Từ vị trí KĐT đi vào trung tâm TP Hà Nội rất thuận tiện, chỉ mất 15 phút chạy xe."

# text = 'Độc quyền suất ngoại giao trực tiếp CĐT: Giá chỉ 25,6tr/m2.\
# TSG Lotus Long Biên: "Mua nhà ở ngay - nhận ngay sổ hồng".\
# * Trả trước chỉ 30% được nhận nhà ở ngay.\
# * Hỗ trợ vay vốn tới 70%GTCH trong 25 năm.\
# * Nhận Sổ ngay sau 45 ngày kể từ khi ký HĐ.\
# * Miễn phí sử dụng dịch vụ đến tháng 10/2021.'

# text = "Ban quản lý dự án Vinhomes Ocean Park kính gửi tới quý khách hàng danh sách biệt thự Shophouse bán mới nhất\
# Cập nhật danh sách Biệt thự shophouse bán tuần 2 tháng 4 đầy đủ nhất, giá tốt nhất thị trường.\
# Tổng hợp biệt thự Hải Âu và San Hô. "

# Thông tin chung cư Vinhomes Ocean Park Gia Lâm\
# Vinhomes Ocean Park Gia Lâm được quy hoạch theo mô hình đô thị Singapore, mang đến cho cư dân một môi trường sống lý tưởng cùng nhiều tiện ích hấp dẫn như công viên thể thao ngoài trời, công viên BBQ... Với giải pháp tài chính linh hoạt, dự án hứa hẹn sẽ là giải pháp đột phá về nhà ở cho người dân và là cơ hội đầu tư đầy tiềm năng.\
# - Tên dự án: Vinhomes Ocean Park (Gia Lâm)\
# - Chủ đầu tư: Công TNHH Đầu tư và Phát triển đô thị Gia Lâm\
# - Nhà thầu thi công: Công ty CP Xây dựng Coteccons\
# - Vị trí: Xã Đa Tốn - Kiêu Kỵ - Dương Xá và một phần thị trấn Trâu Qùy, huyện Gia Lâm, TP. Hà Nội\
# - Tổng diện tích: Khoảng 420ha\
# - Mật độ xây dựng: 19,2%\
# - Diện tích công viên, cây xanh: 62ha\
# - Diện tích hồ điều hòa: 24,5ha\
# - Biển hồ nước mặn: 6,1ha\
# - Quy mô: 2 phân khu gồm phân khu cao tầng và phân khu thấp tầng\
# - Phân khu thấp tầng: gồm 2.300 căn biệt thự với các loại hình\
# + Biệt thự nhà vườn: 168m2\
# + Biệt thự liền kề: 90-130m2\
# + Biệt thự đơn lập: 250-450m2\
# + Biệt thự song lập: 180-250m2\
# + Shophouse: 170-300m2\
content = "Vinhomes Ocean Park Gia Lâm được quy hoạch theo mô hình đô thị Singapore, mang đến cho cư dân một môi trường sống lý tưởng cùng nhiều tiện ích hấp dẫn như công viên thể thao ngoài trời, công viên BBQ... Với giải pháp tài chính linh hoạt, dự án hứa hẹn sẽ là giải pháp đột phá về nhà ở cho người dân và là cơ hội đầu tư đầy tiềm năng.\
- Tên dự án: Vinhomes Ocean Park (Gia Lâm)\
- Chủ đầu tư: Công TNHH Đầu tư và Phát triển đô thị Gia Lâm\
- Nhà thầu thi công: Công ty CP Xây dựng Coteccons\
- Vị trí: Xã Đa Tốn - Kiêu Kỵ - Dương Xá và một phần thị trấn Trâu Qùy, huyện Gia Lâm, TP. Hà Nội\
- Tổng diện tích: Khoảng 420ha\
- Mật độ xây dựng: 19,2%\
- Diện tích công viên, cây xanh: 62ha\
- Diện tích hồ điều hòa: 24,5ha\
- Biển hồ nước mặn: 6,1ha\
- Quy mô: 2 phân khu gồm phân khu cao tầng và phân khu thấp tầng\
- Phân khu thấp tầng: gồm 2.300 căn biệt thự với các loại hình\
+ Biệt thự nhà vườn: 168m2\
+ Biệt thự liền kề: 90-130m2\
+ Biệt thự đơn lập: 250-450m2\
+ Biệt thự song lập: 180-250m2\
+ Shophouse: 170-300m2\
- Phân khu cao tầng: gồm 4 tiểu khu (The Park, The River, The Sea, The Lake) với 66 tòa cao 23-26 tầng, các căn hộ có diện tích 35-85m2, từ 1-3 phòng ngủ"

# text = "thành phố Hà Nội"

# text = "u hướng vì sức khoẻ ở phân khúc nhà hạng sang\nThứ tư, 4/3/2020, 08:00 (GMT+7) \nSunshine Group đầu tư nghìn tỷ cho tòa Aqua Beauty Resort nhằm đáp ứng nhu cầu căn hộ nghỉ dưỡng giới nhà giàu trong bối cảnh môi trường ngày càng ô nhiễm.\n\n\nCuối năm ngoái, Sunshine Group tung ra thị trường tổ hợp tổ hợp resort Sunshine Diamond River nằm trên đường Đào Trí, quận 7 giáp siêu dự án 6 tỷ USD Peninsula. Aqua Beauty Resort là một trong 8 tòa tháp của dự án 1,2 tỷ USD này.\n\nChủ đầu tư không giấu tham vọng biến hạng mục trở thành trung tâm làm đẹp, trị liệu hàng đầu tại Việt Nam. Tòa tháp được thiết kế cảnh quan như một resort tại Bali với không gian cây xanh mặt nước bố trí tận thềm căn hộ. Chủ đầu tư chi cả nghìn tỷ đồng chỉ phát triển hàng trăm tiện ích resort cho toà tháp như: dòng sông nhân tạo dài gần 1.000m hồ chân mây, hơn 40 khu vườn nhiệt đới, hàng chục vườn thiền, hàng chục hồ bơi trên cao, 4.000 vườn nhiệt đới mini trên không, quảng trường ánh sáng, sân trượt băng, khu phố mua sắm trong công viên, chuỗi Bungalow nghỉ dưỡng trên mặt nước.\n\n\n\n\nPhối cảnh dòng sông nhân tạo dài gần 1.000m của Aqua Beauty Resort nhìn từ trên cao.\n\n\nKhách hàng của sản phẩm là giới siêu giàu. Những cư dân thành thị không thể mua cây trồng khắp thành phố, nhưng có thể trồng rất nhiều cây trong khuôn viên dự án chính họ làm ra. \"Tôi không thể bảo vệ cư dân khỏi ô nhiễm ngoài đường, nhưng về nhà, họ sẽ tăng cường sức khỏe qua các tiện ích. Cư dân không có điều kiện du lịch thường xuyên, tôi sẽ khiến nơi ở của họ giống như resort\", vị đại diện chủ đầu tư chia sẻ về lý do Sunshine theo đuổi dòng căn hộ chăm sóc sức khỏe - wellness.\n\nNgoài ra, Sunshine Group đầu tư gần 20 tiện ích chuyên phục vụ trị liệu, nghỉ dưỡng, làm đẹp và thanh lọc cơ thể tại tòa tháp. Trong đó, một khu đại spa hiện đại và quy mô rộng hàng nghìn m2 bố trí tại tầng đế của tòa tháp, với 4 phân khu: aqua spa, formal spa, detox center cùng hệ thống vườn trị liệu. Bên cạnh công nghệ làm đẹp, tòa tháp sở hữu trung tâm thanh lọc cơ thể (detox) quy mô lớn, bao gồm: chuỗi phòng detox thuỷ liệu, quang học và khí detox; chuỗi nhà hàng và cửa hàng chuyên bán và cung cấp các liệu trình detox cho cư dân, trung tâm khí liệu chuyên dạy yoga và phương pháp dưỡng sinh.\n\nThực tế Aqua Beauty Resort là sản phẩm tiếp nối mô hình \"wellness\" trong lĩnh địa ốc tại Việt Nam thời gian qua. Theo giới đầu tư, khái niệm wellness gắn với các không gian sống và làm việc được thiết kế với mục tiêu chú trọng vào sức khỏe thể chất và tinh thần.\n\nManh nha từ một tổ hợp dự án nghỉ dưỡng tại Đà Nẵng hồi những năm 2016-2017, đến 2018 với mô hình này bùng nổ khi lan nhanh sang đến dự án căn hộ cao cấp tại các đô thị lớn. Diamond Island của Kusto Home, The Vista Verde của Capitaland tại khu Đông, Green Star Sky Garden (Hưng Lộc Phát)... những dự án đóng góp cho sự sôi động của phân khúc này trên thị trường TP HCM. Tại Hà Nội, Ecopark Grand - The Island của Tập đoàn Ecopark lại tận dụng lợi thế thiên nhiên sẵn có để tạo dựng một không gian sống đúng chuẩn wellness.\n\nTheo báo cáo Build Well to Live Well, thị trường bất động sản tích hợp các tiện ích hướng đến mục tiêu chăm sóc sức khoẻ (wellness real estate) trên thế giới hiện có giá trị 134 tỷ USD và ước tính sẽ tăng lên 180 tỷ USD vào năm 2022, chiếm khoảng 50% so với ngành công nghiệp xây dựng \"xanh\" trên toàn cầu.\n\nMột báo cáo từ Savills cũng đánh giá sản phẩm \"wellness\" là xu hướng mới trong thị trường bất động sản nghỉ dưỡng toàn cầu với sự xuất hiện của các wellness resort, spa resort. Và tại Việt Nam, mô hình này còn rất mới.\n\n\n\n\nPhối cảnh Detox Center quy mô lớn trong lòng dự án.\n\n\nTiến sĩ Trần Nguyễn Minh Hải - chuyên gia địa ốc Đại học Ngân hàng TP HCM cho rằng thị trường địa ốc đang theo xu hướng phát triển bền vững. Yếu tố khỏe sẽ là tiêu chí ưu tiên khi chọn sản phẩm. Do đó, mô hình căn hộ hướng đến sức khỏe (wellness real estate) đòi hỏi tiềm lực tài chính của mỗi chủ đầu tư. Khi người dân có nhu cầu, thị trường sẽ phải chuyển dịch. \"Đây được xem như tiêu chí quan trọng quyết định việc xuống tiền của nhóm khách hàng trung và thượng lưu\", vị chuyên gia nói.\n\nQuang Minh, một nhà đầu tư tại quận 8, TP HCM, cho rằng xu hướng \"wellness\" nâng tầm giá trị cho mỗi căn hộ và là sản phẩm có tiềm năng đầu tư trong tương lai bởi bắt trúng nhu cầu của những người giàu có.\n\nThanh Hương"



sentences = word_tokenize(content)

tokenize_sentences = []
for paragraph in paragraphs:
    sentences = sent_tokenize(paragraph)
    for sentence in sentences:
#         sentence = self.remove_url(sentence)
        if sentence != '':
            tokenize_sentence = word_tokenize(sentence, format="text")
#             tokenize_sentences.append(tokenize_sentence)
            input_ids = torch.tensor([tokenizer.encode(tokenize_sentence)])  # lấy id của các tokens tương ứng 
            # không dùng tokenize(decode(encode)), text sẽ bị lỗi khi tokenize do conflict với tokenizer mặc định
#             tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(tokenize_sentence))  # lấy các token để đánh tags 

#             outputs = phobert_ner(input_ids).logits
#             predictions = torch.argmax(outputs, dim=2)
    
    
#     for i in [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
#         print(i)
#     print('---------------------------------------------')
    

Token indices sequence length is longer than the specified maximum sequence length for this model (261 > 256). Running this sequence through the model will result in indexing errors


RuntimeError: The expanded size of the tensor (261) must match the existing size (258) at non-singleton dimension 1.  Target sizes: [1, 261].  Tensor sizes: [1, 258]

In [69]:
for sentence in tokenize_sentences:
    list_ids = tokenizer(sentence)['input_ids']

#         if len(list_ids) >= 256:
#             list_ids = list_ids[0:255]
    input_ids = torch.tensor([list_ids])

    tokens = tokenizer.convert_ids_to_tokens(list_ids)
    outputs = phobert_ner(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)
    for i in [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
        print(i)
    print('---------------------------------------------')

('<s>', 'O')
('FPT', 'MCK')
('cho', 'O')
('biết', 'O')
(',', 'O')
('doanh_thu', 'O')
('công_ty', 'O')
('tăng', 'O')
('tại', 'O')
('mọi', 'O')
('thị_trường', 'O')
(',', 'O')
('đặc_biệt', 'O')
('tại', 'O')
('thị_trường', 'O')
('Mỹ', 'O')
('(', 'O')
('+', 'O')
('6@@', 'O')
('0,7', 'O')
('%', 'O')
(')', 'O')
('và', 'O')
('thị_trường', 'O')
('AP@@', 'O')
('AC', 'O')
('(', 'O')
('+', 'O')
('41', 'O')
('%', 'O')
(')', 'O')
('.', 'O')
('</s>', 'O')
---------------------------------------------
('<s>', 'O')
('Doanh_thu', 'O')
('chuyển_đổi', 'O')
('số', 'O')
('trong', 'O')
('3', 'O')
('tháng', 'O')
('đầu', 'O')
('năm', 'O')
('đạt', 'O')
('1.6@@', 'O')
('48', 'O')
('tỷ', 'O')
('đồng', 'O')
(',', 'O')
('tăng_trưởng', 'O')
('9@@', 'O')
('6,2', 'O')
('%', 'O')
('so', 'O')
('với', 'O')
('cùng', 'O')
('kỳ', 'O')
(',', 'O')
('tập_trung', 'O')
('vào', 'O')
('các', 'O')
('công_nghệ', 'O')
('mới', 'O')
('như', 'O')
('Điện_toán', 'O')
('đám_@@', 'O')
('mây', 'O')
('(', 'O')
('tăng', 'O')
('308', 'O')
('%',

In [55]:
outputs

tensor([[[-1.4981, -1.7473, -2.3166, -3.3962, -3.0136,  6.3143],
         [-2.4347, -2.1935, -2.7712, -3.1141, -3.0071,  7.3433],
         [-1.4981, -1.7473, -2.3166, -3.3962, -3.0137,  6.3143]]],
       grad_fn=<AddBackward0>)

In [70]:
content = "Công ty Cổ phần Dầu Thực vật Tường An (mã CK: TAC) đã công bố BCTC quý 1/2022. Cụ thể, doanh thu thuần đạt 1.697 tỷ đồng tăng 7,2% so với cùng kỳ. Tuy nhiên giá vốn hàng bán chiếm tới 94% trong doanh thu khiến lãi gộp chỉ còn hơn 98 tỷ đồng, giảm 55% so với quý 1/2021. Trong kỳ chi phí bán hàng giảm mạnh từ 113,5 tỷ đồng xuống còn hơn 21 tỷ đồng, chi phí QLDN cũng thấp hơn cùng kỳ. Do lãi gộp thấp nên kết quả TAC vẫn báo lãi sau thuế giảm 32% so với cùng kỳ, đạt 53 tỷ đồng – tương đương EPS đạt 1.559 đồng."

In [50]:
sentences = word_tokenize(content)
len(sentences)

227

In [71]:
# content = self.replace_special_character(content)

# split các đoạn văn
paragraphs = [p for p in content.split("\n") if p != '']

tokenize_sentences = []
for paragraph in paragraphs:
    # split sentence with underthesea sent_tokenize
    sentences = sent_tokenize(paragraph)
    for sentence in sentences:
        if sentence != '':
            # tokenize the sentence
            tokenize_sentence = word_tokenize(sentence, format="text")
            tokenize_sentences.append(tokenize_sentence)

for sentence in tokenize_sentences:
    list_ids = tokenizer(sentence)['input_ids']

#         if len(list_ids) >= 256:
#             list_ids = list_ids[0:255]
    input_ids = torch.tensor([list_ids])

    tokens = tokenizer.convert_ids_to_tokens(list_ids)
    outputs = phobert_ner(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)
    for i in [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
        print(i)
    print('---------------------------------------------')

('<s>', 'O')
('Công_ty', 'B-CNAME')
('Cổ_phần', 'I-CNAME')
('Dầu', 'I-CNAME')
('Thực_vật', 'I-CNAME')
('Tường_@@', 'I-CNAME')
('An', 'I-CNAME')
('(', 'O')
('mã', 'O')
('CK', 'O')
(':', 'O')
('TAC', 'MCK')
(')', 'O')
('đã', 'O')
('công_bố', 'O')
('BCTC', 'O')
('quý', 'O')
('1/@@', 'O')
('2022', 'O')
('.', 'O')
('</s>', 'O')
---------------------------------------------
('<s>', 'O')
('Cụ_thể', 'O')
(',', 'O')
('doanh_thu', 'O')
('thuần', 'O')
('đạt', 'O')
('1.6@@', 'O')
('97', 'O')
('tỷ', 'O')
('đồng', 'O')
('tăng', 'O')
('7,2', 'O')
('%', 'O')
('so', 'O')
('với', 'O')
('cùng', 'O')
('kỳ', 'O')
('.', 'O')
('</s>', 'O')
---------------------------------------------
('<s>', 'O')
('Tuy_nhiên', 'O')
('giá', 'O')
('vốn', 'O')
('hàng', 'O')
('bán', 'O')
('chiếm', 'O')
('tới', 'O')
('94', 'O')
('%', 'O')
('trong', 'O')
('doanh_thu', 'O')
('khiến', 'O')
('lãi', 'O')
('gộp', 'O')
('chỉ', 'O')
('còn', 'O')
('hơn', 'O')
('98', 'O')
('tỷ', 'O')
('đồng', 'O')
(',', 'O')
('giảm', 'O')
('55', 'O')
('%'

In [39]:
len(tokenize_sentences)

1

In [161]:
# VncoreNLP 
ner = annotator.ner(text)

ner

[[('Độc_quyền', 'O'),
  ('suất', 'O'),
  ('ngoại_giao', 'O'),
  ('trực_tiếp', 'O'),
  ('CĐT', 'O'),
  (':', 'O'),
  ('Giá', 'O'),
  ('chỉ', 'O'),
  ('25,6', 'O'),
  ('tr', 'O'),
  ('/', 'O'),
  ('m2', 'O'),
  ('.', 'O')],
 [('TSG', 'O'),
  ('Lotus_Long_Biên', 'B-PER'),
  (':', 'O'),
  ('"', 'O'),
  ('Mua', 'O'),
  ('nhà', 'O'),
  ('ở', 'O'),
  ('ngay', 'O'),
  ('-', 'O'),
  ('nhận', 'O'),
  ('ngay', 'O'),
  ('sổ', 'O'),
  ('hồng', 'O'),
  ('"', 'O'),
  ('.', 'O')],
 [('*', 'O'),
  ('Trả', 'O'),
  ('trước', 'O'),
  ('chỉ', 'O'),
  ('30%', 'O'),
  ('được', 'O'),
  ('nhận', 'O'),
  ('nhà', 'O'),
  ('ở', 'O'),
  ('ngay', 'O'),
  ('.', 'O')],
 [('*', 'O'),
  ('Hỗ_trợ', 'O'),
  ('vay', 'O'),
  ('vốn', 'O'),
  ('tới', 'O'),
  ('70%', 'O'),
  ('GTCH', 'O'),
  ('trong', 'O'),
  ('25', 'O'),
  ('năm', 'O'),
  ('.', 'O')],
 [('*', 'O'),
  ('Nhận_Sổ', 'O'),
  ('ngay', 'O'),
  ('sau', 'O'),
  ('45', 'O'),
  ('ngày', 'O'),
  ('kể', 'O'),
  ('từ', 'O'),
  ('khi', 'O'),
  ('ký', 'O'),
  ('HĐ', 'O'),
 

In [4]:
!pip install underthesea

     |████████████████████████████████| 7.6 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 235 kB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 580 kB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 15.9 MB/s eta 0:00:01
